
#Neuroguard, Machine learning for intracranial pressure monitoring

##Specific objectives
1. development a algoritm

##Sections of the project
1. Set up the environment
2. Download relevant databases and resources
3. Preprocessing 
4. Create new development space with SAM
5. Assign SAM configuration
6. Testing on random images for segmentation
7. Check out our WEB page


<a id="1.setup"></a>
#1. Set Up the environment


In [ ]:
!gdown --id 1RplOOrelIOycetRtEpoYrLmFZLtsJZUX
!7z x SAM_workshop_data2.zip

import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

import torch
import torchvision
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

!{sys.executable} -m pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!{sys.executable} -m pip install lvis
!{sys.executable} -m pip install lvis-api/.
!{sys.executable} -m pip install tqdm

from copy import deepcopy
import logging
from collections import defaultdict
from collections import OrderedDict

from lvis.lvis import LVIS
import pycocotools.mask as mask_utils
import datetime

from tqdm import tqdm
import json

from lvis import LVIS, LVISVis, LVISResults, LVISEval

#2. Download relevant databases and resources

In [ ]:
!wget -O kaggle.json https://raw.githubusercontent.com/aarevalom0/NeuroGuard/refs/heads/main/kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# Files of Kaggle competitions
!kaggle competitions download -c aptos2019-blindness-detection
!unzip aptos2019-blindness-detection.zip -d ./dataset

#3. Preprocesing images

In [ ]:
imagen = cv2.imread('/content/dataset/test_images/006efc72b638.png')
imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10, 5))
plt.title("Imagen Original")
plt.imshow(imagen)
plt.axis("off")
plt.show()

# 4. Create new development space with SAM

In [ ]:
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)

mask_generator = SamAutomaticMaskGenerator(sam)
masks = mask_generator.generate(imagen)

#Assign SAM configuration

In [ ]:
def mostrar_anotaciones(anotaciones):
    if len(anotaciones) > 0:
        anotaciones_ordenadas = sorted(anotaciones, key=(lambda x: x['area']), reverse=True)
        ax = plt.gca()
        ax.set_autoscale_on(False)
        img = np.ones((anotaciones_ordenadas[0]['segmentation'].shape[0], anotaciones_ordenadas[0]['segmentation'].shape[1], 4))
        img[:,:,3] = 0
        for anotacion in anotaciones_ordenadas:
            m = anotacion['segmentation']
            color_mask = np.concatenate([np.random.random(3), [0.35]])  # Generar un color aleatorio con una transparencia del 35%
            img[m] = color_mask
        ax.imshow(img)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Imagen Original")
plt.imshow(imagen)
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Segmentación SAM")
plt.imshow(imagen)
mostrar_anotaciones(masks)
plt.axis("off")

plt.show()

#Watershep

In [ ]:
gray = cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY)

gray = cv2.GaussianBlur(gray, (5, 5), 0)

_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
sure_bg = cv2.dilate(opening, kernel, iterations=3)

dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

_, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown == 255] = 0

imagen_watershed = imagen.copy()
cv2.watershed(imagen_watershed, markers)
imagen_watershed[markers == -1] = [255, 0, 0]

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Imagen Original")
plt.imshow(imagen)
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Segmentación Watershed")
plt.imshow(imagen_watershed)
plt.axis("off")

plt.show()